In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Code used to load data
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Create target object and call it y
ames_housing_data_path = '../input/housing/train.csv'

ames_housing = pd.read_csv(ames_housing_data_path)

ames_housing_data = ames_housing.dropna(axis ='columns')

ames_housing_data.describe()

y = ames_housing_data['SalePrice']

In [ ]:
# Create features and call it X

X = ames_housing.drop(columns = ['Misc Feature','Fence','Pool QC','Fireplace Qu','Alley'])

In [ ]:
# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.1)

In [ ]:
# Specify Model
ames_housing_model = DecisionTreeRegressor(random_state=1)


# Fit Model
#since we have categorical data, we wont train directly but say

numerical_cols = train_X.select_dtypes(include='number').columns.to_list()

categorical_cols = train_X.select_dtypes(exclude='number').columns.to_list()

numerical_pipe = Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler()) ])

categorical_pipe = Pipeline([ ('imputer', SimpleImputer(strategy='most_frequent')), ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

ct = ColumnTransformer(remainder='drop',
                       transformers=[('numerical', numerical_pipe, numerical_cols), ('categorical', categorical_pipe, categorical_cols)])
ames_housing_model = Pipeline([('transformer', ct), ('predictor', RandomForestRegressor())])

ames_housing_model.fit(train_X, train_y)

In [ ]:
predicted_y = ames_housing_model.predict(val_X)

In [ ]:
mean_squared_error(predicted_y, val_y, squared=False)

In [ ]:
y_output = ames_housing_model.predict(X)

In [ ]:
output = pd.DataFrame({'Id':X.PID, 
                    'SalePrice': y_output})
output.to_csv('submission.csv',index=False)